In [2]:
import pandas as pd
import time
from datetime import datetime
import requests
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

In [3]:
#Data load parameters

polygon_api_key = 'RMnfdtr9nmyTjXjgbNJeX_I5pIcowZpl'

load_ticker = 'TQQQ'
load_interval = '5'
load_time_unit = 'minute'
load_start_date = '2019-05-01' #first trading day 5 years ago
load_end_date = '2024-04-01'
max_days_til_sold = 25 * 78

# Parameters in percentage; How much % change from the prev day closing price to initiate a buy, and how much % to exit the position
price_dec_pct_buy = -2
price_inc_pct_sell = 2

#Load data from Polygon.io
url = f'https://api.polygon.io/v2/aggs/ticker/{load_ticker}/range/{load_interval}/{load_time_unit}/{load_start_date}/{load_end_date}?adjusted=true&sort=asc&limit=50000&apiKey={polygon_api_key}'
r = requests.get(url)
data = r.json()

close = []
time = []

while 'next_url' in data.keys():
    for d in data['results']:
        close.append(d['c'])
        time.append(datetime.fromtimestamp(int(str(d['t'])[:-3])))

    url = data['next_url']+f"&apiKey={polygon_api_key}"
    r = requests.get(url)
    data = r.json()

for d in data['results']:
    close.append(d['c'])
    time.append(datetime.fromtimestamp(int(str(d['t'])[:-3])))

dff = pd.DataFrame({'time': time, 'close': close});

dff['time'] = pd.to_datetime(dff['time'])
df = dff.copy()


# Define function to check if time is between 9:30 am to 4:00 pm

def check_trading_hours(dte):
    if dte.hour >= 9 and dte.hour < 16:  # Checking if the hour is between 9 and 16 (4 PM)
        if dte.hour == 9:
            if dte.minute > 30:  # If hour is 9, check if minute is > 30
                return True
            else:
                return False
        else:
            return True
    elif dte.hour == 16 and dte.minute == 0:
        return True
    else:
        return False

# Apply the function to the datetime column to create a new column indicating trading hours
df['is_trading_hours'] = df['time'].apply(check_trading_hours)

#df.to_excel("tradinghours.xlsx")
# Count the number of trading hour times
num_trading_hours = df['is_trading_hours'].sum()

# BUY TRACK: when the price decreased x% or more than previous day's closing price

# Calculate the date of the day before
df['previous_day'] = df['time'].apply(lambda x: x - pd.Timedelta(days=1))
df['previous_day'] = pd.to_datetime(df['previous_day'])
df['previous_day_4pm'] = df['previous_day'].apply(lambda x: x.replace(hour=16, minute=0))

# Merge with itself to find the price of the day before at 4 pm
merged_df = pd.merge(df, df, left_on='previous_day_4pm', right_on='time', suffixes=('', '_previous'))

# Calculate the decrease percentage compared to the price of the day before at 4 pm
merged_df['price_decrease_percentage'] = (merged_df['close'] - merged_df['close_previous']) / merged_df['close_previous'] * 100

# Make a new dataframe and clean it up
buy_track_temp = merged_df.copy()
buy_track_temp=buy_track_temp.drop(columns=['previous_day', 'time_previous', 'is_trading_hours_previous', 'previous_day_previous', 'previous_day_4pm_previous'])

# Make a new dataframe that tracks past buy signals
buy_track = buy_track_temp[buy_track_temp['price_decrease_percentage'] <= price_dec_pct_buy]
buy_track = buy_track[buy_track['is_trading_hours'] == True]
df['included_in_buy_track'] = df['time'].isin(buy_track['time'])

df2 = df[df['is_trading_hours'] == True].reset_index(drop= True)

df2['day'] = df2.time.dt.date
x = pd.DataFrame(df2.groupby(['day']).min()['close']).reset_index()
x['max'] = df2.groupby(['day']).max()['close'].tolist()
x['day_before'] = df2.groupby('day')['close'].agg(['last']).reset_index().shift(1)['last'].tolist()
x.columns = ['day', 'min', 'max', 'day_before']
x['min_pct'] = ((x['min']-x['day_before'])/x['day_before'])*100
x['max_pct'] = ((x['max']-x['day_before'])/x['day_before'])*100
x['year'] = [x['day'][i].year for i in range(x.shape[0])]

fig = go.Figure(data=[go.Histogram(name = 'minimum pct',
                                    x=x['min_pct'], 
                                    opacity= 0.5,
                                    xbins=dict(start=-10.0,
                                              end=10.0,
                                              size=0.5))])
fig.add_trace(go.Histogram(name = 'maximum pct',
                            x=x['max_pct'],
                            opacity= 0.5,
                            xbins=dict(start=-10.0,
                                              end=10.0,
                                              size=0.5)))
fig.update_layout(barmode='overlay',
                  title = f'Overall Min/Max Histogram versus Day Before',
                  yaxis_title = 'count')

fig.show()

for year, group in x.groupby(['year']):   
    fig = go.Figure(data=[go.Histogram(name = 'minimum pct',
                                       x=group['min_pct'], 
                                       opacity= 0.5,
                                       xbins=dict(start=-10.0,
                                                  end=10.0,
                                                  size=0.5))])
    fig.add_trace(go.Histogram(name = 'maximum pct',
                               x=group['max_pct'],
                               opacity= 0.5,
                               xbins=dict(start=-10.0,
                                                  end=10.0,
                                                  size=0.5)))
    fig.update_layout(barmode='overlay',
                      title = f'{year[0]} Min/Max Histogram versus Day Before',
                      yaxis_title = 'count')
    fig.show()